In [1]:
# -*- coding: utf-8 -*-
"""
@author: Alain Plantec

Voici un squelette possible.
Vous devez programmez les classes contenues dans ce squelette.
Les fonctions et leurs paramètres ainsi que les variables contenues dans le classes
ont du sens par rapport à une version programmée pour la préparation du projet.
Votre version sera forcément différente.
Donc, vous pouvez ajouter/retirer des variables et/ou des fonctions et/ou des paramètres.

"""
try:  # import as appropriate for 2.x vs. 3.x
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox

from sokobanXSBLevels import *
from enum import Enum

"""
Direction :
    Utile pour gérer le calcul des positions pour les mouvements
"""
class Direction(Enum):
    Up = 1
    Down = 2
    Left = 3
    Right = 4

"""
Position :
    - stockage de coordoannées x et y,
    - vérification de x et y par rapport à une matrice
    - calcule de position relative à partir d'un offset (un décalage) et une direction 
"""
class Position(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return 'Position(' + str(self.x) + ',' + str(self.y) + str(')') 

    def getX(self):
        return self.x

    def getY(self):
        return self.y

    # retoune la position vers la direction #direction en tenant compte de l'offset
    #   Position(3,4).positionTowards(Direction.Right, 2) == Position(5,4)
    def positionTowards(self, direction, offset):
        if direction==1:
            self.y-=offset
        if direction==2:
            self.y+=offset
        if direction==3:
            self.x-=offset
        if direction==4:
            self.x+=offset

    # Retourne True si les coordonnées sont valides dans le wharehouse
    def isValidInWharehouse(self, wharehouse):
        return wharehouse.isPositionValid(self)

    # Convertit le receveur en une position correspondante dans un Canvas
    def asCanvasPositionIn(self, elem):
        lx = self.getX() * elem.getWidth() 
        ly = self.getY() * elem.getHeight()
        return Position(lx, ly)

"""
WharehousePlan : Plan de l'entrepot pour stocker les éléments.
    Les éléments sont stockés dans une matrice (#rawMatrix)
"""
class WharehousePlan(object):
    def __init__(self):
        # la matrice d'Elem
        self.rawMatrix = []

    def appendRow(self, row):
        self.rawMatrix.append(row)
 
    def at(self, position):
        return self.rawMatrix[position.x][position.y]
    
    def atPut(self, position, elem):
        self.rawMatrix[position.x][position.y]=elem
        
    def isPositionValid(self, position):
        a=True
        m=0
        for i in range(len(self.rawMatrix)):
            if len(self.rawMatrix[i])>m:
                m=len(self.rawMatrix[i])
        if position.x<0 or position.x>m:
            a=False
        if position.y<0 or position.y>len(self.rawMatrix)-1:
            a=False
        return a;
        

    def hasFreePlaceAt(self, position):
        return self.at(position).isFreePlace()

    def asXsbMatrix(self):
        return xsbMatrix(self.rawMatrix)
    
"""
Floor :
    Représente une case vide de la matrice
    (pas de None dans la matrice)
"""
class Floor(object):
    def __init__(self):
        None
    def isMovable(self):
        return False
    def canBeCovered(self):
        return True
    def xsbChar(self):
        return ' '
    def isFreePlace(self):
        return True
    
"""
Goal :
    Représente une localisation à recouvrir d'un BOX (objectif du jeu).
    Le déménageur doit parvenir à couvrir toutes ces cellules à partir des caisses.
    Un Goal est static, il est toujours déssiné en dessous :
        Le zOrder est assuré par le tag du create_image (tag='static')
        et self.canvas.tag_raise("movable","static") dans Level
"""
class Goal(object):
    def __init__(self, canvas, position,ms):
        self.image=tk.PhotoImage(file='goal.png')
        ms[position.y][position.x]=self.image
        canvas.create_image((position.x+0.5)*64,(position.y+0.5)*64,image=ms[position.y][position.x],tag='static')

    def isMovable(self):
        return False

    def getHeight(self):
        return self.height
    
    def getWidth(self):
        return self.width

    def canBeCovered(self):
        return True
        
    def xsbChar(self):
        return '.'

    def isFreePlace(self):
        return False

"""
Wall : pour délimiter les murs
    Le déménageur ne peut pas traverser un mur.
    Un Wall est static, il est toujours déssiné en dessous :
        Le zOrder est assuré par le tag du create_image (tag='static')
        et self.canvas.tag_raise("movable","static") dans Level
"""
class Wall(object):
    def __init__(self, canvas, position,ms):
        
        self.image = tk.PhotoImage(file='wall.png')
        ms[position.y][position.x] = self.image
        canvas.create_image((position.x+0.5)*64,(position.y+0.5)*64,image=ms[position.y][position.x],tag='static')

    def getHeight(self):
        return self.height
    
    def getWidth(self):
        return self.width

    def isMovable(self):
        return False

    def canBeCovered(self):
        return False

    def xsbChar(self):
        return '#'

    def isFreePlace(self):
        return False

"""
Box : Caisse à déplacer par le déménageur.
    Etant donné qu'une caisse doit être déplacé, le canvas et la matrice sont necessaires pour
    reconstruire l'image et mettre en oeuvre sont déplacement (dans le canvas et dans la matrice)
    Un Box est "movable", il est toujours déssiné au dessus des objets "static" :
        Le zOrder est assuré par le tag du create_image (tag='movable')
        et self.canvas.tag_raise("movable","static") dans Level
    Un Box est représenté differemment (image différente) suivant qu'il se situe sur un emplacement marqué par un Goal ou non.
 """
class Box(object):
    def __init__(self, canvas,wharehouse, position, onGoal,m):
       
        self.x=position.x
        self.y=position.y
        if onGoal==0:
            self.image=tk.PhotoImage(file='box.png')
        else:
            self.image=tk.PhotoImage(file='boxOnTarget.png')
        
        m[position.y][position.x]=self.image
        self.boxId = canvas.create_image((position.x+0.5)*64,(position.y+0.5)*64,image=m[position.y][position.x],tag='movable')
        self.m=m
        self.canvas=canvas
        
    def getHeight(self):
        return self.height
    
    def getWidth(self):
        return self.width

    def isMovable(self):
        return True

    def canBeCovered(self):
        return False

    def moveTowards(self, direction,mat):
        y=self.y
        x=self.x
        self.mat=mat
        if direction == 'Up':
            self.y = self.y - 1
        elif direction == 'Down':
            self.y = self.y + 1
        elif direction == 'Left':
            self.x = self.x - 1
        else:
            self.x = self.x + 1

        self.m[self.y][self.x] = self.m[y][x]

        if self.mat[y][x]=='#':
            self.m[y][x] = None
        
        self.canvas.delete(self.boxId)
       
        self.boxId = self.canvas.create_image(self.x * 64, self.y * 64, image=self.m[self.y][self.x], anchor=tk.NW, tag="movable")
            
 
    def xsbChar(self):
        if self.under.isFreePlace(): return '$'
        else: return '*'

    def isFreePlace(self):
        return False

    def startGoalCoveredAnimation(self):
        self.image=tk.PhotoImage(file='box.png')
        self.m[self.y][self.x] = self.image
        self.canvas.delete(self.boxId)
        self.boxId = self.canvas.create_image(self.x * 64, self.y * 64, image=self.image, anchor=tk.NW, tag="movable")
        
 
    def cleanUpAnimation(self):
        None
  
    def goalCoveredAnimation(self):
        self.image=tk.PhotoImage(file='boxOnTarget.png')
        self.m[self.y][self.x] = self.image
        self.canvas.delete(self.boxId)
        self.boxId = self.canvas.create_image(self.x * 64, self.y * 64, image=self.image, anchor=tk.NW, tag="movable")
        
        

 
"""
Mover : C'est  le déménageur.
    La classe Mover met en oeuvre la logique du jeu dans #canMove et #moveTowards.
    Etant donné qu'un Mover se déplace, le canvas et la matrice sont necessaires pour
    reconstruire l'image et mettre en oeuvre sont déplacement (dans le canvas et dans la matrice)
    Un Mover est "movable", il est toujours déssiné au dessus des objets "static" :
        Le zOrder est assuré par le tag du create_image (tag='movable')
        et self.canvas.tag_raise("movable","static") dans Level
    Un Box est représenté differemment (image différente) suivant la direction de déplacement (même si le déplacement s'avère impossible).
"""
class Mover(object):
    def __init__(self, canvas, wharehouse, position, onGoal,m):  
        self.x=position.x
        self.y=position.y
        if onGoal==0:
            self.image = tk.PhotoImage(file='playerDown.png')
        else:
            self.image = tk.PhotoImage(file='playerDown.png') 
        m[position.y][position.x] = self.image
        self.playerId = canvas.create_image((position.x+0.5)*64,(position.y+0.5)*64, image=m[position.y][position.x],tag='movable')
        self.m=m
        self.canvas=canvas
        self.cercle=None
        self.rayon=10
        
        
         
    def getHeight(self):
        return self.height
    
    def getWidth(self):
        return self.width

    def isMoveable(self):
        return True

    def moveInCanvas(self, direction,mat):
        y=self.y
        x=self.x
        self.mat=mat
        if direction == 'Up':
            self.y = self.y - 1
        elif direction == 'Down':
            self.y = self.y + 1
        elif direction == 'Left':
            self.x = self.x - 1
        else:
            self.x = self.x + 1

        self.m[self.y][self.x] = self.m[y][x]

        if self.mat[y][x]=='#':
            self.m[y][x] = None
        
        self.canvas.delete(self.playerId)
       
        self.playerId = self.canvas.create_image(self.x * 64, self.y * 64, image=self.m[self.y][self.x], anchor=tk.NW, tag="movable")
            
        

    """
        Retourne True si le Mover peut se déplacer dans la direction demandée.
        Le calcul necessite de voir l'élément adjacent mais aussi l'élément suivant (offset de 2)
    """
    def canMove(self, direction, mate,wharehouse,ms):
        x = self.x
        y = self.y
        rep=False
       
        if direction == 'Up':
            
            if mate[y-1][x]=='$' or mate[y-1][x]=='*': 
                if mate[y-2][x]!='#' and mate[y-2][x]!='$' and mate[y-2][x]!='*':
                    rep=True
                    
                    wharehouse.at(Position(y-1,x)).moveTowards(direction,mate)
                    temp=wharehouse.at(Position(y-1,x))
                    wharehouse.atPut(Position(y-2,x),temp)
                    t=mate[y-1][x]
                    
                    if mate[y-1][x]=='*':
                        mate[y-1][x]="."
                        temp2=Goal(wharehouse.at(Position(y-1,x)).canvas,Position(x,y-1),ms)
                        if mate[y-2][x]==" ":
                            wharehouse.at(Position(y-1,x)).startGoalCoveredAnimation()
                            t='$'
                    else:
                        mate[y-1][x]=" "
                        temp2=Floor()
                        if mate[y-2][x]==".":
                            wharehouse.at(Position(y-1,x)).goalCoveredAnimation()
                            t='*'
                        
                    wharehouse.atPut(Position(y-1,x),temp2)
                    mate[y-2][x]=t    
            
                    
            else:
                if mate[y-1][x]!='#': 
                    rep=True   
                    
        elif direction == 'Down':
            if mate[y+1][x]=='$'or mate[y+1][x]=='*': 
                if mate[y+2][x]!='#' and mate[y+2][x]!='$' and mate[y+2][x]!='*':
                    rep=True
                    
                    wharehouse.at(Position(y+1,x)).moveTowards(direction,mate)
                    temp=wharehouse.at(Position(y+1,x))
                    wharehouse.atPut(Position(y+2,x),temp)
                    t=mate[y+1][x]
                    
                    if mate[y+1][x]=='*':
                        mate[y+1][x]="."
                        temp2=Goal(wharehouse.at(Position(y+1,x)).canvas,Position(x,y+1),ms)
                        if mate[y+2][x]==" ":
                            wharehouse.at(Position(y+1,x)).startGoalCoveredAnimation()
                            t='$'
                    else:
                        mate[y+1][x]=" "
                        temp2=Floor()
                        if mate[y+2][x]==".":
                            wharehouse.at(Position(y+1,x)).goalCoveredAnimation()
                            t='*'
                    wharehouse.atPut(Position(y+1,x),temp2) 
                    mate[y+2][x]=t
            else:
                if mate[y+1][x]!='#': 
                    rep=True 
                    
        elif direction == 'Left':
            if mate[y][x-1]=='$' or mate[y][x-1]=='*': 
                if mate[y][x-2]!='#' and mate[y][x-2]!='$' and mate[y][x-2]!='*':
                    rep=True
                    
                    wharehouse.at(Position(y,x-1)).moveTowards(direction,mate)
                    temp=wharehouse.at(Position(y,x-1))
                    wharehouse.atPut(Position(y,x-2),temp)
                    t=mate[y][x-1]
                    
                    if mate[y][x-1]=='*':
                        mate[y][x-1]="."
                        temp2=Goal(wharehouse.at(Position(y,x-1)).canvas,Position(x-1,y),ms)
                        if mate[y][x-2]==" ":
                            wharehouse.at(Position(y,x-1)).startGoalCoveredAnimation()
                            t='$'
                    else:
                        mate[y][x-1]=" "
                        temp2=Floor()
                        if mate[y][x-2]==".":
                            wharehouse.at(Position(y,x-1)).goalCoveredAnimation()
                            t='*'
                    wharehouse.atPut(Position(y,x-1),temp2)
                    mate[y][x-2]=t
            else:
                if mate[y][x-1]!='#': 
                    rep=True 
                    
        else:
            
            if mate[y][x+1]=='$' or mate[y][x+1]=='*': 
                if mate[y][x+2]!='#' and mate[y][x+2]!='$' and mate[y][x+2]!='*':
                    rep=True
                    
                    wharehouse.at(Position(y,x+1)).moveTowards(direction,mate)
                    temp=wharehouse.at(Position(y,x+1))
                    wharehouse.atPut(Position(y,x+2),temp)
                    t=mate[y][x+1]
                    
                    if mate[y][x+1]=='*':
                        mate[y][x+1]="."
                        temp2=Goal(wharehouse.at(Position(y,x+1)).canvas,Position(x+1,y),ms)
                        if mate[y][x+2]==" ":
                            wharehouse.at(Position(y,x+1)).startGoalCoveredAnimation()
                            t='$'
                    else:
                        mate[y][x+1]=" "
                        temp2=Floor()
                        if mate[y][x+2]==".":
                            wharehouse.at(Position(y,x+2)).goalCoveredAnimation()
                            t='*'
                    wharehouse.atPut(Position(y,x+1),temp2)
                    mate[y][x+2]=t
                    
                   
            else:
                if mate[y][x+1]!='#': 
                    rep=True 
        
        return rep

    """
        Pour le déplacement, il faut penser à déplacer éventuellemnt le Box et ensuite déplacer le Mover
    """
    def moveTowards(self, direction):
        None

    """
        Le Mover est représenté differemment suivant la direction de déplacement
    """
    def setupImageForDirection(self, direction):
        if direction == 'Up':
            self.image = tk.PhotoImage(file='playerUp.png')
            
        elif direction == 'Down':
            self.image = tk.PhotoImage(file='playerDown.png')
        elif direction == 'Left':
            self.image = tk.PhotoImage(file='playerLeft.png')
        else:
            self.image = tk.PhotoImage(file='playerRight.png')
        self.canvas.delete(self.playerId)
        self.playerId = self.canvas.create_image(self.x * 64, self.y * 64, image=self.image, anchor=tk.NW, tag="movable")
        
            

    """
        Pour le déplacement :
            - image changée en fonction de la direction
            - si on ne peut pas se déplacer dans cette direction -> abandon
            - sinon, bin le Mover est déplacé
    """
    def push(self, direction):
        self.setupImageForDirection(direction)
        if not self.canMove(direction):
            self.startImpossiblePushAnimation()
            return
        self.moveTowards(direction)

    def xsbChar(self):
        if self.under.isFreePlace(): return '@'
        else: return '+'

    def isFreePlace(self):
        return False

    def startImpossiblePushAnimation(self):
        None

    def cleanUpAnimation(self):
        if self.cercle!=None:
            self.canvas.delete(self.cercle)
            self.cercle=None
        self.rayon=10

    def impossiblePushAnimation(self):
        x=(self.x+0.5)*64-self.rayon/2
        y=self.y*64-self.rayon
        if self.cercle!=None:
            self.canvas.delete(self.cercle)
            self.rayon=self.rayon+5
          
        self.cercle=self.canvas.create_oval(x,y,x+self.rayon,y+self.rayon,fill='yellow',width=0) 
        
        if self.rayon==50:
            self.rayon=5

            
    
"""
    Le jeux avec tout ce qu'il faut pour dessiner et stocker/gérer la matrice d'éléments
    
"""
class Level(object):
    def __init__(self, root, xsbMatrix):
        self.root = root
        self.wharehouse = WharehousePlan()
        self.xsbMatrix=xsbMatrix

        # calcul des dimensions de la matrice
        self.nbrows = len(xsbMatrix)
        self.nbcolumns = 0

        for line in xsbMatrix:
            nbc = len(line)
            if nbc > self.nbcolumns:
                self.nbcolumns = nbc

        self.height = self.nbrows * 64 
        self.width = self.nbcolumns * 64 
 
        self.canvas = tk.Canvas(self.root, width=self.width, height=self.height, bg="gray")
        self.canvas.pack()

        self.initWharehouseFromXsb(xsbMatrix)
        self.root.bind("<Key>", self.keypressed)

    def initWharehouseFromXsb(self,xsbMatrix):
        # legend :
        #   '#' = wall,  '$' = box, '.' = goal, '*' = box on goal, '@' = mover, '+' = mover on goal, '-' = floor, ' ' = floor
        self.staticMatrix = []
        self.movableMatrix = []
        for lineIdx in range(self.nbrows):
            self.staticMatrix.append([])
            self.movableMatrix.append([])
            for elemIdx in range(self.nbcolumns):
                self.staticMatrix[lineIdx].append(None)
                self.movableMatrix[lineIdx].append(None)
                
        y = 0
        for lineIdx in range(len(xsbMatrix)):
            x = 0
            lst=[]
            for elemIdx in range(len(xsbMatrix[lineIdx])):
                e = xsbMatrix[lineIdx][elemIdx]
                if e == '#':
                    self.wall=Wall(self.canvas,Position(x,y),self.staticMatrix)
                    lst.append(self.wall)
                elif e=='@':
                    self.mover=Mover(self.canvas,self.wharehouse,Position(x,y),0,self.movableMatrix)
                    lst.append(Floor())
                elif e=='+':
                    self.mover=Mover(self.canvas,self.wharehouse,Position(x,y),1,self.movableMatrix)
                    lst.append(Floor())
                elif e=='$':
                    self.box=Box(self.canvas,self.wharehouse,Position(x,y),0,self.movableMatrix)
                    lst.append(self.box)
                elif e=='*':
                    self.box=Box(self.canvas,self.wharehouse,Position(x,y),1,self.movableMatrix)
                    lst.append(self.box)
                elif e=='.':
                    self.goal=Goal(self.canvas,Position(x,y),self.staticMatrix)
                    lst.append(self.goal)
    
                else:
                    lst.append(Floor())
                x = x + 1
            self.wharehouse.appendRow(lst)
            y = y + 1
            x = 0
           
        self.canvas.tag_raise("movable","static")
         
    def keypressed(self, event):
        if self.mover.canMove(event.keysym,self.xsbMatrix,self.wharehouse,self.staticMatrix,):
            self.mover.moveInCanvas(event.keysym,self.xsbMatrix)
            self.mover.setupImageForDirection(event.keysym)
            self.mover.cleanUpAnimation()
            
        else:
            self.mover.impossiblePushAnimation()
        

class Sokoban(object):
    '''
    Main Level class
    '''

    def __init__(self):
        self.root = tk.Tk()
        self.root.resizable(False, False)
        self.root.title("Sokoban")
        print('Sokoban: ' + str(len(SokobanXSBLevels)) + ' levels')
        self.level = Level(self.root, SokobanXSBLevels[99])
        #self.level = Level(self.root, [
            #['-','-','$','+','$','.','-','.','.','.','.','-','-','.','.','-','-','.','-'] ])
        #self.level = Level(self.root, [ ['@'] ])
 
    def play(self):
        self.root.mainloop()


Sokoban().play()

Sokoban: 101 levels
